In [2]:
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'C:\\Users\\unign\\.wdm\\drivers\\chromedriver\\win64\\128.0.6613.84\\chromedriver-win32/chromedriver.exe'

In [3]:
from selenium import webdriver
browser = webdriver.Chrome()
url = 'https://google.com'
browser.get(url)

In [4]:
from selenium.webdriver.common.by import By

browser.find_element(By.CLASS_NAME, 'uU7dJb').text

'대한민국'

In [5]:
url = 'https://www.yes24.com/Product/category/bestseller?CategoryNumber=001&sumgb=06'
browser.get(url)

In [6]:
# #1개 수집할 때
# browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
# #전체 링크 데이터
# datas  = browser.find_elements(By.CLASS_NAME, 'gd_name')

# for i in datas:
#     print(i.get_attribute('href'))


In [7]:
import time


link_list = []
for i in range(1,4):
    print(f"{i}페이지 수집")
    
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
        
    time.sleep(1)

print(link_list)

1페이지 수집
2페이지 수집
3페이지 수집
['https://www.yes24.com/Product/Goods/102457889', 'https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/130500819', 'https://www.yes24.com/Product/Goods/104687898', 'https://www.yes24.com/Product/Goods/109145992', 'https://www.yes24.com/Product/Goods/95721612', 'https://www.yes24.com/Product/Goods/129602366', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/131930939', 'https://www.yes24.com/Product/Goods/131931234', 'https://www.yes24.com/Product/Goods/128917300', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/132560712', 'https://www.yes24.com/Product/Goods/126642929', 'https://www.yes24.com/Product/Goods/130033773', 'https://www.yes24.com/Product/Goods/132556345', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/96547408', 'https://www.yes24.com/Product/Goods/124043802

In [17]:
# 상세 페이지로 이동하기
#for link in link_list:
    #browser.get(link)
    
browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]

print(ranking)
print(ranking_weeks)

베스트 국내도서 1위
국내도서 1위 2주


In [19]:
import pymysql
import re
from datetime import datetime

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='@Ab1480718',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for link in link_list:
        browser.get(link)
    
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
    
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2024-08-26"
            
            
        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        review = int(review.replace(",",""))
        
        
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",",""))
        
        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))
        
            
        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
                parts = full_text.split(" | ")
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part))
            except:
                ranking = 0
                
            print(ranking)
            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0
            
    
    ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
    ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]
    
    sql = """INSERT INTO Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )"""
    
    cur.execute(sql,( title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
    conn.commit()
    
    time.sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


ValueError: invalid literal for int() with base 10: '예약판매'